In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .appName("Tier Analysis") \
    .getOrCreate()

23/08/24 17:07:56 WARN Utils: Your hostname, isoyeon-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.75 instead (on interface en0)
23/08/24 17:07:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/24 17:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data_spark = spark.read.csv('./../data1.csv', header=True, inferSchema=True)

In [7]:
data_spark.createOrReplaceTempView("games")

In [9]:
data_spark

DataFrame[tier: string, match_id: string, team_id: int, position: string, kills: int, deaths: int, assists: int, win: boolean, champion_name: string, champion_id: int, patch: double]

tier,match_id,team_id,position,kills,deaths,assists,win,champion_name,champion_id,patch

In [10]:
win_rate_query = """
SELECT tier, COUNT(*) AS total_games, SUM(CASE WHEN win = 'yes' THEN 1 ELSE 0 END) AS wins, (SUM(CASE WHEN win = 'yes' THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS win_rate
FROM games
GROUP BY tier
"""

win_rate_result = spark.sql(win_rate_query)
win_rate_result.show()

+-----------+-----------+----+--------+
|       tier|total_games|wins|win_rate|
+-----------+-----------+----+--------+
|   DIAMONDI|       9070|4535|    50.0|
|   BRONZEIV|       8900|4450|    50.0|
|    BRONZEI|       8000|4000|    50.0|
|GRANDMASTER|       8720|4360|    50.0|
|  EMERALDIV|       9120|4560|    50.0|
|  BRONZEIII|       7360|3680|    50.0|
|PLATINUMIII|       8670|4335|    50.0|
|     IRONIV|       8530|4265|    50.0|
|  EMERALDII|       8310|4155|    50.0|
|    GOLDIII|       8440|4220|    50.0|
|  DIAMONDII|       8700|4350|    50.0|
|   EMERALDI|       9770|4885|    50.0|
|     MASTER|       9490|4745|    50.0|
|      IRONI|       7580|3790|    50.0|
|  PLATINUMI|       8950|4475|    50.0|
|   BRONZEII|       9220|4610|    50.0|
| PLATINUMII|       8530|4265|    50.0|
|     GOLDIV|       8520|4260|    50.0|
| EMERALDIII|       8560|4280|    50.0|
| CHALLENGER|       8930|4465|    50.0|
+-----------+-----------+----+--------+
only showing top 20 rows



- 밴률을 어떻게 찾지 , 라인별로

In [11]:
from pyspark.sql.functions import col

# 픽률 및 승률 계산
query = """
SELECT
    tier,
    champion_name,
    COUNT(*) AS total_games,
    SUM(CASE WHEN win = 'yes' THEN 1 ELSE 0 END) AS wins,
    (COUNT(*) / (SELECT COUNT(*) FROM games)) * 100 AS pick_rate,
    (SUM(CASE WHEN win = 'yes' THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS win_rate
FROM games
GROUP BY tier, champion_name
"""

result = spark.sql(query)

# 결과 출력
result.show()

+-----------+-------------+-----------+----+--------------------+------------------+
|       tier|champion_name|total_games|wins|           pick_rate|          win_rate|
+-----------+-------------+-----------+----+--------------------+------------------+
|   BRONZEII|         Fizz|         34|  12|0.012769953051643192|35.294117647058826|
|     IRONII|        Kaisa|        239| 113| 0.08976525821596244| 47.28033472803347|
|PLATINUMIII|        Swain|         71|  35| 0.02666666666666667| 49.29577464788733|
|     IRONII|       Ezreal|        251| 117| 0.09427230046948357|46.613545816733065|
|  EMERALDII|       Akshan|         28|  12|0.010516431924882629|42.857142857142854|
|  SILVERIII|       Aatrox|        112|  51|0.042065727699530514|45.535714285714285|
|PLATINUMIII|         Sion|         33|  20|0.012394366197183098| 60.60606060606061|
|  DIAMONDIV|         Fizz|         22|   9|0.008262910798122065|40.909090909090914|
| PLATINUMIV|         Sona|         10|   6|0.003755868544600939|